In [ ]:
# MongoDB vs ClickHouse

# MongoDB

In [8]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [18]:
from pymongo import MongoClient
uri = 'mongodb://localhost:27017/'
client = MongoClient(uri, directConnection=True)


In [10]:
client.list_database_names()

['Bookmarks',
 'Rating',
 'Review',
 'UserMovie',
 'admin',
 'config',
 'local',
 'test']

In [426]:
# client.drop_database('Bookmarks')

## Вставка данных

### User-Movie 

In [189]:
user_movie = client.UserMovie

In [66]:
user_movie_df = pd.read_csv('user_data.csv')

In [68]:
user_movie_df = user_movie_df[['user_id', 'movie_id', 'ids']]

In [209]:
user_movie_df_1 = user_movie_df[:1000000]

In [210]:
%%time
step = 3000
for i in range(0, user_movie_df_1.shape[0], step):
    batch = user_movie_df_1[i:i+step].copy()
    user_movie.UserMovieCollection.insert_many(batch.to_dict('records'))

CPU times: user 14.5 s, sys: 7.44 s, total: 21.9 s
Wall time: 1min 6s


In [73]:
user_movie_df.shape

(5000000, 3)

In [69]:
%%time
user_movie.collection.insert_many(user_movie_df.to_dict('records'))

CPU times: user 59.4 s, sys: 29.2 s, total: 1min 28s
Wall time: 7min 53s


### Bookmarks 

In [19]:
bookmarks = client.Bookmarks
bookmarks_df = pd.read_csv('bookmarks.csv')
bookmarks_df = bookmarks_df[['user_id', 'movie_id']]
bookmarks_df.head()

,user_id,movie_id
0,62e1d789-0699-4a1a-a20a-41a71966a71e,a1e1f25e-02f0-4768-bad6-6aca3792e569
1,ba7e90d2-e3bc-48ad-b08a-3f63f41c00c5,3e0dcf60-c5be-46cc-a86d-c3b6a416a92e
2,5f365c30-8f86-4d21-a9fa-873d5d319d13,0bb5a969-ccc6-4cab-8aff-da455b50c5fd
3,40b0a62e-df6d-4e30-b6d9-d8bd40e404af,35c40d1e-6304-41df-b39f-8c98b26bcb4d
4,0772e536-6814-4136-a2b8-be096af3aad2,18d7f6f3-a5b1-49f2-bea2-fb7bff210537


In [72]:
bookmarks_df.shape

(100000, 2)

In [20]:
bookmarks_new = bookmarks_df.to_dict('records')

In [415]:
%%time
bookmarks.BookmarksCollection.insert_many(bookmarks_new)

CPU times: user 605 ms, sys: 202 ms, total: 806 ms
Wall time: 13.8 s


#### Вставка по одному элементу на небольшом объеме данных 

In [429]:
%%time
for item in bookmarks_new:
    bookmarks.BookmarksCollection.insert_one(item)

CPU times: user 49.1 s, sys: 6.79 s, total: 55.9 s
Wall time: 4min 3s


In [21]:
%%time
for item in bookmarks_new:
    bookmarks.BookmarksCollection.insert_one(item)

NotPrimaryError: not primary, full error: {'topologyVersion': {'processId': ObjectId('63b6b3e25d9236735d558de2'), 'counter': 4}, 'ok': 0.0, 'errmsg': 'not primary', 'code': 10107, 'codeName': 'NotWritablePrimary', 'lastCommittedOpTime': Timestamp(1672918323, 1), '$clusterTime': {'clusterTime': Timestamp(1672918323, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1672918323, 1)}

### Rating 

In [98]:
rating = client.Rating
rating_df = pd.read_csv('user_rating2.csv')
rating_df = rating_df[['user_id', 'movie_id', 'rating']]

In [99]:
rating_df.shape

(5000000, 3)

In [108]:
rating_df.head()

,user_id,movie_id,rating
0,871c1108-5cd3-4919-9574-b1ae57d9a180,4321308e-0fa0-4ece-9382-356f697ba823,6
1,5294b3ff-0ccd-4b28-b9f7-0a1313a032f7,09cde2d6-23a7-4015-a05c-7b4b259920fd,8
2,1164ea22-8d62-477e-8e52-50faaedf52df,b55eb1da-f942-4b06-aaf7-e4b974a57c58,9
3,99b34ff9-cb4d-4657-8e68-da120616a59d,a9f975ec-2e43-4f5e-b222-3d2ee8a0311c,7
4,b85619a4-56b4-493c-9ee0-3fdc8f888b0c,a4219fdf-b4d2-4498-871c-968b21b0d355,4


In [230]:
rating_df_1 = rating_df[:1000000]

In [231]:
%%time
step = 3000
for i in range(0, rating_df_1.shape[0], step):
    batch = rating_df_1[i:i+step].copy()
    rating.RatingCollection.insert_many(batch.to_dict('records'))

CPU times: user 10.2 s, sys: 724 ms, total: 10.9 s
Wall time: 35.7 s


In [232]:
rating_df_2 = rating_df[1000000:2000000]

In [233]:
%%time
step = 3000
for i in range(0, rating_df_2.shape[0], step):
    batch = rating_df_2[i:i+step].copy()
    rating.RatingCollection.insert_many(batch.to_dict('records'))

CPU times: user 11.9 s, sys: 2.96 s, total: 14.8 s
Wall time: 50.4 s


### Review 

In [102]:
review = client.Review
review_df = pd.read_csv('user_review2.csv')
review_df = review_df[['user_id', 'movie_id', 'text', 'date']]

In [103]:
review_df['id'] = [str(uuid.uuid4()) for _ in range(100000)]

In [104]:
review_df.shape

(100000, 5)

In [190]:
%%time
review.ReviewCollection.insert_many(review_df.to_dict('records'))

CPU times: user 1.04 s, sys: 225 ms, total: 1.26 s
Wall time: 8.2 s


In [119]:
review_df.head()

,user_id,movie_id,text,date,id
0,94f9c9a4-ee56-466a-9d21-28b0a3575996,ce666a8b-4b08-4aff-851a-70e9617a7cf3,CYPkLIEueVksobAOZgSbOgXjioMSqxRNTKPalTUDptKnil...,2023-01-03 01:01:55.755298,2d02f694-7aa8-4ff1-860a-24d986e5c350
1,1498a393-ebd7-428b-beba-678f83a189e4,174c3caa-5977-4cfe-be63-de037814f65a,dakvIRQStvFQycRDnBDhinBHNriawIp ijKQYBxnag tIn...,2023-01-03 01:01:55.755315,3285f490-1e66-4194-b51a-ed63b051024b
2,6963789f-be66-464c-97ca-f0c502a2d01e,54d39f4a-3fce-4d2d-ab5c-7c490abd5878,VJHhacgjqHQSxCyvZbJSAtYbjpbiFlCEBaHsNCxbQEzVZQ...,2023-01-03 01:01:55.755316,baf177c2-8640-4056-b169-c86b4a13a1d4
3,9f1bdd38-6030-4f86-8585-9a3706ed0c5c,ff7a3d8f-8145-4af1-9861-b1a20cd0a34f,gzsdrFBZAoFghWodCjyDFvLIWwIMojlmxtuFZdcZKBFwPP...,2023-01-03 01:01:55.755317,3e4b4d19-4a1c-4bff-89d0-79a61cc5280b
4,f65e223a-771d-4238-94d5-418cb5c294a1,3e83f62b-79a0-4edb-92e7-938c7effb078,LtBswJYOMyAAqXquklqfdilSqCvgzalpLQoBASmKxLDLpd...,2023-01-03 01:01:55.755319,4878b82b-3a8a-4052-b139-aac80e6c74b0


## Чтение данных 

### Чтение всех данных

In [235]:
%%time
extracted_data = user_movie.UserMovieCollection.find({})
us_m_extracted = list(extracted_data)

CPU times: user 2.24 s, sys: 951 ms, total: 3.19 s
Wall time: 8.56 s


In [236]:
%%time
extracted_data = bookmarks.BookmarksCollection.find({})
book_extracted = list(extracted_data)


CPU times: user 122 ms, sys: 83.6 ms, total: 205 ms
Wall time: 973 ms


In [237]:
%%time
extracted_data = rating.RatingCollection.find({})
rating_extracted = list(extracted_data)


CPU times: user 6.23 s, sys: 5.28 s, total: 11.5 s
Wall time: 30.8 s


In [238]:
%%time
extracted_data = review.ReviewCollection.find({})
review_extracted = list(extracted_data)


CPU times: user 196 ms, sys: 224 ms, total: 420 ms
Wall time: 3.12 s


### Средний рейтинг по конкретному фильму 

In [258]:
check_rating = rating_df[:2000000]

In [326]:
# для проверки корректности полученных данных
check_rating.groupby(['movie_id'])['movie_id'].count()[-1]

229

In [325]:
check_rating.groupby(['movie_id'])['rating'].sum()[-1]

1080

In [337]:
%%time

agr = [{'$match': {"movie_id": "fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab"}}, 
       {'$group': {'_id': 1, 'all': {'$sum':'$rating'}, 'count': {'$sum': 1}}}]
rating_res = list(rating.RatingCollection.aggregate(agr))
average_rating = rating_res[0]['all'] / rating_res[0]['count']

CPU times: user 6.07 ms, sys: 2.79 ms, total: 8.87 ms
Wall time: 491 ms


In [338]:
print('Result for movie "fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab" =', average_rating)

Result for movie "fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab" = 4.716157205240175


### Закладки конкретного пользователя

In [349]:
bookmarks_df.groupby(['user_id'])[['movie_id']].count().sort_values('movie_id').tail()

,movie_id
user_id,
53ffafc8-ff85-44ff-b6df-a499d153f634,2
75f213f0-069e-4681-87dc-0abbe2bd36ec,2
580fc4bd-a929-4fb1-8b9e-7dbf0357efd7,2
f7c48507-46ce-4184-a6a0-56a3de84879a,2
53f1cc4f-65a6-4e33-a8e7-c22b737d94a0,2


In [348]:
bookmarks_df[bookmarks_df['user_id'] == '53f1cc4f-65a6-4e33-a8e7-c22b737d94a0']

,user_id,movie_id
4407,53f1cc4f-65a6-4e33-a8e7-c22b737d94a0,ab601b06-2991-48a2-aa2d-b8d01749abd1
99995,53f1cc4f-65a6-4e33-a8e7-c22b737d94a0,5104d275-e0f2-494b-8f0d-1c504a477b8e


In [357]:
%%time

extracted_data = bookmarks.BookmarksCollection.find({'user_id':'53f1cc4f-65a6-4e33-a8e7-c22b737d94a0'})
book_extracted = list(extracted_data)

CPU times: user 2.85 ms, sys: 2.76 ms, total: 5.61 ms
Wall time: 26.7 ms


### Количество положительных отметок у конкретного фильма

In [371]:
%%time
search_res = len(list(rating.RatingCollection.find({"movie_id": "fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab", 
                                                    'rating': {'$gt': 6}})))

CPU times: user 5.22 ms, sys: 4.42 ms, total: 9.64 ms
Wall time: 726 ms


In [372]:
search_res

67

# Clickhouse

## Вставка 

In [466]:
from clickhouse_driver import Client

client = Client(host='localhost', settings={"use_numpy": True})

In [388]:
import warnings
warnings.filterwarnings('ignore')

In [381]:
client.execute('CREATE DATABASE IF NOT EXISTS ugc ON CLUSTER company_cluster')

[array(['clickhouse-node1', '9000', '0', '', '3', '0'], dtype='<U21'),
 array(['clickhouse-node3', '9000', '0', '', '2', '0'], dtype='<U21'),
 array(['clickhouse-node4', '9000', '0', '', '1', '0'], dtype='<U21'),
 array(['clickhouse-node2', '9000', '0', '', '0', '0'], dtype='<U21')]

In [384]:
client.execute('CREATE TABLE IF NOT EXISTS ugc.UserMovie ON CLUSTER company_cluster (user_id UUID, movie_id UUID, ids UUID) Engine=MergeTree() ORDER BY user_id')


[array(['clickhouse-node1', '9000', '0', '', '3', '0'], dtype='<U21'),
 array(['clickhouse-node3', '9000', '0', '', '2', '0'], dtype='<U21'),
 array(['clickhouse-node2', '9000', '0', '', '1', '0'], dtype='<U21'),
 array(['clickhouse-node4', '9000', '0', '', '0', '0'], dtype='<U21')]

In [506]:
# client.execute('DROP TABLE ugc.Bookmarks;')

[]

In [507]:
client.execute('CREATE TABLE IF NOT EXISTS ugc.Rating ON CLUSTER company_cluster (user_id UUID, movie_id UUID, rating UInt8) Engine=MergeTree() ORDER BY user_id')
client.execute('CREATE TABLE IF NOT EXISTS ugc.Review ON CLUSTER company_cluster (user_id UUID, movie_id UUID, text LONGTEXT, date DateTime) Engine=MergeTree() ORDER BY user_id')
client.execute('CREATE TABLE IF NOT EXISTS ugc.Bookmarks ON CLUSTER company_cluster (user_id UUID, movie_id UUID) Engine=MergeTree() ORDER BY user_id')


[array(['clickhouse-node1', '9000', '0', '', '3', '0'], dtype='<U21'),
 array(['clickhouse-node3', '9000', '0', '', '2', '0'], dtype='<U21'),
 array(['clickhouse-node4', '9000', '0', '', '1', '0'], dtype='<U21'),
 array(['clickhouse-node2', '9000', '0', '', '0', '0'], dtype='<U21')]

In [394]:
bookmarks_df = pd.read_csv('bookmarks.csv', dtype='object')
bookmarks_df = bookmarks_df[['user_id', 'movie_id']]

user_movie_df = pd.read_csv('user_data.csv', dtype='object')
user_movie_df = user_movie_df[['user_id', 'movie_id', 'ids']]

rating_df = pd.read_csv('user_rating2.csv')
rating_df = rating_df[['user_id', 'movie_id', 'rating']]

review_df = pd.read_csv('user_review2.csv')
review_df = review_df[['user_id', 'movie_id', 'text', 'date']]

In [400]:
query = 'INSERT INTO ugc.Rating VALUES'
step = 3000

In [401]:
%%time
for i in range(0, rating_df.shape[0], step):
    batch = rating_df[i:i+step].copy()
    client.insert_dataframe(query, batch)

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

...

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

CPU times: user 20.7 s, sys: 1.92 s, total: 22.6 s
Wall time: 44.5 s


In [403]:
query = 'INSERT INTO ugc.Review VALUES'
step = 3000

In [404]:
%%time
for i in range(0, review_df.shape[0], step):
    batch = review_df[i:i+step].copy()
    client.insert_dataframe(query, batch)

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

CPU times: user 592 ms, sys: 148 ms, total: 740 ms
Wall time: 2.57 s


In [508]:
query = 'INSERT INTO ugc.Bookmarks VALUES'
step = 3000

In [511]:

%%time
for i in range(0, bookmarks_df.shape[0], step):
    batch = bookmarks_df[i:i+step].copy()
    client.insert_dataframe(query, batch)

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

CPU times: user 502 ms, sys: 44.3 ms, total: 547 ms
Wall time: 1.16 s


#### Вставка по одному элементу на небольшом объеме данных 

In [488]:
bookmarks_df_1 = bookmarks_df[:10000]

In [490]:
%%time
query = 'INSERT INTO ugc.Bookmarks VALUES'
step = 1
for i in range(0, bookmarks_df_1.shape[0], step):
#     batch = bookmarks_df_1[i:i+step].copy()
    client.insert_dataframe(query, bookmarks_df_1[i:i+step])
    

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

...

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column


CPU times: user 30 s, sys: 7.26 s, total: 37.2 s
Wall time: 2min


## Чтение данных 

### Чтение всех данных 

In [409]:
%%time
data = client.execute('SELECT * FROM ugc.UserMovie')

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

CPU times: user 43.6 s, sys: 37.3 s, total: 1min 20s
Wall time: 2min 49s


In [411]:
%%time
warnings.filterwarnings('ignore', category=DeprecationWarning)
data = client.execute('SELECT * FROM ugc.Bookmarks')

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column


CPU times: user 440 ms, sys: 90 ms, total: 530 ms
Wall time: 908 ms


In [412]:
%%time
data = client.execute('SELECT * FROM ugc.Review')

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

CPU times: user 553 ms, sys: 133 ms, total: 686 ms
Wall time: 1.05 s


In [413]:
%%time
data = client.execute('SELECT * FROM ugc.Rating')

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUI

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column


CPU times: user 45.8 s, sys: 38.7 s, total: 1min 24s
Wall time: 3min 2s


### Средний рейтинг по конкретному фильму 

In [505]:
%%time
data = client.execute("SELECT avg(rating) FROM ugc.Rating WHERE movie_id=='{}'".format('fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab'))
data

CPU times: user 2.47 ms, sys: 2.45 ms, total: 4.92 ms
Wall time: 130 ms


[array([4.89876265])]

### Закладки конкретного пользователя

In [512]:
%%time
data = client.execute("SELECT movie_id FROM ugc.Bookmarks WHERE user_id=='{}'".format('53f1cc4f-65a6-4e33-a8e7-c22b737d94a0'))
data

    

NumPy support is not implemented for UUID. Using generic column
NumPy support is not implemented for UUID. Using generic column


CPU times: user 4.49 ms, sys: 4.49 ms, total: 8.98 ms
Wall time: 32.9 ms


[array([UUID('ab601b06-2991-48a2-aa2d-b8d01749abd1')], dtype=object),
 array([UUID('5104d275-e0f2-494b-8f0d-1c504a477b8e')], dtype=object)]

### Количество положительных отметок у конкретного фильма

In [513]:
%%time
data = client.execute("SELECT COUNT(rating) FROM ugc.Rating WHERE movie_id=='{}' and rating>=6".format('fffb0bdd-ab3e-4e4b-a9d4-0870989b85ab'))
data

CPU times: user 2.57 ms, sys: 4.59 ms, total: 7.16 ms
Wall time: 212 ms


[array([377], dtype=uint64)]